Загружаем и импортируем нужные библиотеки.

In [ ]:
!pip install pymorphy2==0.8
!pip install spacy-udpipe

In [3]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from typing import List, Tuple
import random
import re
import time
from tqdm import tqdm
from lxml import etree

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
import spacy_udpipe
spacy_udpipe.download("ru")

Downloaded pre-trained UDPipe model for 'ru' language


Функция для считывания и предобработки датасета. На выходе получаем 1) тексты, из которых удаленны ссылки и никнеймы, 2) лемматизированный твит и 3) их соответвующие теги.

In [63]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    nlp = spacy_udpipe.load("ru")
    lemmatized_texts = []
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in tqdm(database.getchildren()):
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        raw_text = str(column.text).strip()
                        # removing links or nicknames
                        middle_text = re.sub('(http://.+)|(.*//t.co/.+)|([_@]+\w+)',' ', raw_text)
                        new_text = re.sub('  ',' ', middle_text)
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                if new_text != ' ': 
                  lemmatized_text = ' '.join([token.lemma_ for token in nlp(new_text)])
                  lemmatized_texts.append(lemmatized_text)
                else:
                  lemmatized_texts.append(new_text)
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, lemmatized_texts, labels

In [64]:
texts, lemmatized_texts, labels = load_sentirueval_2016('bank_train_2016.xml')

100%|██████████| 9393/9393 [00:45<00:00, 206.33it/s]


In [65]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 9392, number of labels is 9392.


In [66]:
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

neutral =>  
neutral =>  
neutral =>  
positive => За полгода благодаря ипотеке от Россельхозбанка жилищные условия улучшили больше 300 красноярских семей. Филиал выдал на эти цели в общей с
neutral =>  
neutral =>  
neutral =>  
neutral =>  А-Ч привязывается к карте,не к счету. Вопрос, какие смс не пришли.Можем проверить отправку.Киньте № вашего тел. на social .ru
negative => RT :  Под новые санкции от ЕС попадают Сбербанк и #ВТБ Порошенко! Почему у нас эти вражеские банки до сих пор работа…
positive =>  
neutral =>  
negative => Опять «порадовал»  Платёжка получена банком в 15:28 и так и не проведена. Ну никак не хотят работать!
neutral =>  
neutral =>  
negative => RT :  Просто журнал не пользуется вашими услугами, никогда не искал тарифы на вашем сайте и не был дезинформирован вашим…
neutral =>  
positive => Интернет-банк Сбербанка обновился #ЧитайМеняАяТебя
neutral =>  
negative =>  
neutral =>  


In [67]:
positive_tweets = [lemmatized_texts[idx] for idx in range(len(lemmatized_texts)) if labels[idx] == 'positive']
negative_tweets = [lemmatized_texts[idx] for idx in range(len(lemmatized_texts)) if labels[idx] == 'negative']

In [68]:
for cur in positive_tweets[:5]: print(cur)

главное чтобы банк СБЕР и ВТБ ! ! !
 
 
 
в наш сбербанк прийти американец и просить сфоткаться с он у банкомат ; D я васто быть в шок и на фотка полюбому получиться смешной xD


In [69]:
for cur in negative_tweets[:5]: print(cur)

обязательно про сбербанк писать ! временами хлестко интерать магазин финт выкручивать )
втб и сбер точно . вопрос - блокировка операция или запрет на кредит ?
Канада ввести санкция против Газпромбанк , ВЭБ , Новатэк и российский оборонный компания
RT : если Сбер и ВТБ попасть под санкция , быть жоп
RT : # Канада ввести санкция против 10 оборонный и сырьевой компания и банк #РФ . среди они , в частность , " # Газпромбанк " , " # Внешэк …


In [70]:
vectorizer = CountVectorizer(tokenizer=word_tokenize)

In [71]:
vectorizer.fit(lemmatized_texts)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x7fc377791b70>,
                vocabulary=None)

In [72]:
print(vectorizer.get_feature_names()[0:20])

['!', '#', '$', '%', '&', "''", '(', ')', '*^', '*бесится*', '*трипл-смайлик-крик', '+', '+3,49', '+300', '+5тысяча', '+74955102933', ',', '-', '-in', '-а']


In [73]:
print(len(vectorizer.get_feature_names()))

5901


In [77]:
X = vectorizer.transform(lemmatized_texts)

In [78]:
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [80]:
print(lemmatized_texts[10])

дело - на 1 миллиард грн уйти в минус Райффайзен Банк Аваль и быть докапитализировать украинский " дочка " ВТб


In [81]:
print(X[10])

  (0, 17)	1
  (0, 57)	1
  (0, 326)	2
  (0, 616)	1
  (0, 808)	1
  (0, 969)	1
  (0, 974)	1
  (0, 1245)	1
  (0, 1473)	1
  (0, 1546)	1
  (0, 1636)	1
  (0, 1687)	1
  (0, 1994)	1
  (0, 2713)	1
  (0, 2730)	1
  (0, 2851)	1
  (0, 4353)	1
  (0, 5377)	1
  (0, 5390)	1


In [83]:
print(vectorizer.get_feature_names()[2321])

коммерческий


In [84]:
print(vectorizer.get_feature_names()[96])

140


In [85]:
print(vectorizer.get_feature_names()[16])

,


In [86]:
from sklearn.feature_extraction.text import TfidfTransformer

In [87]:
transformer = TfidfTransformer().fit(X)

In [88]:
X_transformed = transformer.transform(X)

In [89]:
print(X_transformed[10])

  (0, 5390)	0.26552533657725413
  (0, 5377)	0.2784019288526588
  (0, 4353)	0.22530409747514774
  (0, 2851)	0.1267017928520832
  (0, 2730)	0.2926000714886608
  (0, 2713)	0.19408307002189207
  (0, 1994)	0.11718194377453123
  (0, 1687)	0.2683281863361969
  (0, 1636)	0.33107009927712666
  (0, 1546)	0.24710317233675994
  (0, 1473)	0.3067982141246628
  (0, 1245)	0.1396693825562318
  (0, 974)	0.10643310874771426
  (0, 969)	0.16744198682407113
  (0, 808)	0.12471448354023115
  (0, 616)	0.2522367756872963
  (0, 326)	0.31860488772068735
  (0, 57)	0.222831287184296
  (0, 17)	0.14884177002193988


In [94]:
print(vectorizer.get_feature_names()[1000])

ввод


In [96]:
print(vectorizer.get_feature_names()[779])

аэрофлот


In [97]:
tokens_with_IDF = list(zip(vectorizer.get_feature_names(), transformer.idf_))

In [98]:
for feature, idf in tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.662923 => !
4.051895 => #
6.621359 => $
5.885040 => %
6.020586 => &
9.454573 => ''
5.287908 => (
4.844415 => )
9.454573 => *^
9.454573 => *бесится*
9.454573 => *трипл-смайлик-крик
9.454573 => +
9.454573 => +3,49
9.454573 => +300
9.454573 => +5тысяча
9.454573 => +74955102933
3.069378 => ,
4.250566 => -
9.454573 => -in
9.454573 => -а


In [99]:
sorted_tokens_with_IDF = sorted(tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [100]:
for feature, idf in sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => ''
9.454573 => *^
9.454573 => *бесится*
9.454573 => *трипл-смайлик-крик
9.454573 => +
9.454573 => +3,49
9.454573 => +300
9.454573 => +5тысяча
9.454573 => +74955102933
9.454573 => -in
9.454573 => -а
9.454573 => -банка
9.454573 => -депозит
9.454573 => -й
9.454573 => -клик
9.454573 => -клика
9.454573 => -кликом
9.454573 => -круто
9.454573 => -лидер
9.454573 => -пересадочный


In [101]:
from sklearn.feature_selection import SelectPercentile, chi2

In [102]:
selector = SelectPercentile(chi2, percentile=20)

In [103]:
selector.fit(X_transformed, labels)

SelectPercentile(percentile=20, score_func=<function chi2 at 0x7fc31a0502f0>)

In [104]:
selected_tokens_with_IDF = [tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]

In [105]:
print(len(selected_tokens_with_IDF))

1180


In [106]:
for feature, idf in selected_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.662923 => !
4.051895 => #
5.885040 => %
5.287908 => (
4.844415 => )
9.454573 => *^
9.454573 => +
9.454573 => +5тысяча
3.069378 => ,
4.250566 => -
9.454573 => -депозит
3.158385 => .
8.761426 => ...
9.049108 => 0,5
7.950495 => 00
6.363530 => 1
7.950495 => 1,5
6.851883 => 10
7.012226 => 100
9.454573 => 103


In [107]:
selected_and_sorted_tokens_with_IDF = sorted(selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [108]:
for feature, idf in selected_and_sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => *^
9.454573 => +
9.454573 => +5тысяча
9.454573 => -депозит
9.454573 => 103
9.454573 => 164
9.454573 => 19,2
9.454573 => 20,9
9.454573 => 2009
9.454573 => 212,4
9.454573 => 220,3
9.454573 => 262,8
9.454573 => 3,35
9.454573 => 3,6
9.454573 => 3,7
9.454573 => 324
9.454573 => 344
9.454573 => 402
9.454573 => 5-
9.454573 => 500тысяча
